In [1]:

import torch
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"


This version features the desired income generation function; a fitted function is used as a decent placeholder to determine consumption. There are many aspects (such as optimization and personal perception of theta) which will be required in future but are currently not utilized. With the model in its current arrangement, the agents stop trading and stabillize after ~7 time steps. I have included a commented-out section at the bottom which uses matplotlib pyplot if you wish to get a quick look.
The current version of the config.yaml file is not formatted correctly and should not be used as is.

In [2]:
model = dgl_ptm.PovertyTrapModel(model_identifier='testwithpseudo')

In [3]:
model.set_model_parameters(default=False, **{'step_target': 2,'steering_parameters': {'npath':'./agent_data.zarr',
                            'epath':'./edge_data', 
                            'ndata':['all_except',['a_table']],
                            'edata':['all'],
                            'mode':'xarray',
                            'wealth_method':'singular_transfer',
                            'income_method':'default',
                            'capital_update_method':'default',
                            'consume_method':'default',
                            'perception_method':'default',
                            'tech_gamma': torch.tensor([0.3,0.35,0.45]),
                            'tech_cost': torch.tensor([0,0.15,0.65]),
                            'adapt_m':torch.tensor([0,0.5,0.9]),
                            'adapt_cost':torch.tensor([0,0.25,0.45]),
                            'depreciation': 0.6,
                            'discount': 0.95,
                            'm_theta_dist': {'type':'multinomial','parameters':[[0.01 ,0.1, 0.79, 0.1],[0.1, 0.5, 0.8, 1]],'round':False,'decimals':None},
                            'del_prob':0.05,
                            'ratio':0.1,
                            'weight_a':0.69,
                            'weight_b':35, 
                            'truncation_weight':1.0e-10,
                            'step_type': 'custom'}})

print(model.number_agents)  
print(model.steering_parameters)                        

100
{'npath': './agent_data.zarr', 'epath': './edge_data', 'ndata': ['all_except', ['a_table']], 'edata': ['all'], 'mode': 'xarray', 'wealth_method': 'singular_transfer', 'income_method': 'default', 'capital_update_method': 'default', 'consume_method': 'default', 'perception_method': 'default', 'tech_gamma': tensor([0.3000, 0.3500, 0.4500]), 'tech_cost': tensor([0.0000, 0.1500, 0.6500]), 'adapt_m': tensor([0.0000, 0.5000, 0.9000]), 'adapt_cost': tensor([0.0000, 0.2500, 0.4500]), 'depreciation': 0.6, 'discount': 0.95, 'm_theta_dist': {'type': 'multinomial', 'parameters': [[0.01, 0.1, 0.79, 0.1], [0.1, 0.5, 0.8, 1]], 'round': False, 'decimals': None}, 'del_prob': 0.05, 'ratio': 0.1, 'weight_a': 0.69, 'weight_b': 35, 'truncation_weight': 1e-10, 'step_type': 'custom'}


In [4]:
model.initialize_model()

In [5]:
model.run()



performing step 1 of 2
creating bidirectional link between nodes 97 (src) and tensor([16]) (dst)
created 1 of 1 links requested
running perception update
running consume update
running income update
performing step 2 of 2
running capital update
dst node 90 is an end node
no FoF link possible for src dst nodes 16 and 90.
created 0 of 1 links requested
running perception update
running consume update
running income update


In [43]:
'''
# try to see something happening
import zarr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation



agent_data = zarr.open('/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/agent_data.zarr', mode='r')
alpha=pd.DataFrame(agent_data['alpha'])
net_trade=pd.DataFrame(agent_data['net_trade'])
k=pd.DataFrame(agent_data['wealth'])

fig=plt.figure()
plt.xlabel("α")
plt.ylabel("$k_{t}$")
#xlims=[0.8,OGdata.loc[TimeSample,"alpha"].max()]
xlims=[0.8,1.5]

ylims=[0,10]
#ylims=[0,OGdata.loc[TimeSample,"k_t"].max()]

fig.suptitle(f"k$_{{{'t'}}}$ v.s. α at Time Step {0}")



def animate3(i,alpha, k,net_trade, xlim,ylim):

    plt.clf()
    fig.suptitle(f"k$_{{{'t'}}}$ v.s. α at Time Step {i}")
    plt.xlabel("α")
    plt.ylabel("$k_{t}$")
    pointart=plt.scatter(alpha[i], k[i])
    plt.xlim(xlim)
    plt.ylim(ylim)
    snapshot=str(f"Net Traded:{net_trade[i].mean()}")
    fig.text(0.05, 0.021, snapshot,fontsize=10, bbox=dict(fc="white", ec="white"))





points=plt.scatter(alpha[0], k[0])
    #hue="i_a", 
    #palette=["b", "r","#7f7f7f"], 

plt.xlim(xlims)
plt.ylim(ylims)

anim3 = animation.FuncAnimation(fig, animate3, frames=k.shape[1], interval=2000, blit=False, fargs=(alpha,k,net_trade,xlims,ylims,)) 



plt.show()
'''

'\n# try to see something happening\nimport zarr\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport matplotlib.animation as animation\n\n\n\nagent_data = zarr.open(\'/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/agent_data.zarr\', mode=\'r\')\nalpha=pd.DataFrame(agent_data[\'alpha\'])\nnet_trade=pd.DataFrame(agent_data[\'net_trade\'])\nk=pd.DataFrame(agent_data[\'wealth\'])\n\nfig=plt.figure()\nplt.xlabel("α")\nplt.ylabel("$k_{t}$")\n#xlims=[0.8,OGdata.loc[TimeSample,"alpha"].max()]\nxlims=[0.8,1.5]\n\nylims=[0,10]\n#ylims=[0,OGdata.loc[TimeSample,"k_t"].max()]\n\nfig.suptitle(f"k$_{{{\'t\'}}}$ v.s. α at Time Step {0}")\n\n\n\ndef animate3(i,alpha, k,net_trade, xlim,ylim):\n\n    plt.clf()\n    fig.suptitle(f"k$_{{{\'t\'}}}$ v.s. α at Time Step {i}")\n    plt.xlabel("α")\n    plt.ylabel("$k_{t}$")\n    pointart=plt.scatter(alpha[i], k[i])\n    plt.xlim(xlim)\n    plt.ylim(ylim)\n    snapshot=str(f"Net Traded:{net_trade[i].mean()}")\n    fig.text(0.05, 0.021, s